In [1]:
###  EXTRACT DATASET     8 image per class as traing data and 2 image per class as test data

import os
import numpy as np
import cv2
image_name_for_training=[]
labels_name_for_training=[]
image_name_for_testing=[]
labels_name_for_testing=[]
training_image_per_class=8
train_horses = "ORL_dataset" 

for name in os.listdir(train_horses):
    i=0
    data_path = os.path.join(train_horses,name)
    for name_img in os.listdir(data_path):
        dir_path=os.path.join(data_path,name_img)
        if(training_image_per_class>i):
            image_name_for_training += [dir_path]
            labels_name_for_training+=[name]
            i+=1
        else:
            image_name_for_testing += [dir_path]
            labels_name_for_testing+=[name]

In [2]:
### start making 2D PCA  #### it directly give eivenvector, and pca traing test data, mean
import numpy.linalg as LA
def eigenvector_2DPCA(list_img):
    mean_gray=np.zeros((112,92))
    for name_img in list_img :
        gray=cv2.imread(name_img,0)
        mean_gray+=gray
    mean_gray=mean_gray/len(list_img)
    image_matrix_for_traning=np.zeros((len(list_img),112,92))
    scatter_matrix=np.zeros((92,92))
    i=0
    for name_img in list_img:
        img_matrix=cv2.imread(name_img,0)
        img_matrix=img_matrix-mean_gray
        scatter_matrix+=np.matmul(img_matrix.T,img_matrix)
        image_matrix_for_traning[i,:,:]=img_matrix
        i+=1
    eigen_val,eigen_vector=LA.eig(scatter_matrix)
    sort_eigenval=np.sort(eigen_val)
    x=np.sum(sort_eigenval[76:])/np.sum(sort_eigenval)  # this give the 95% energy 
    final_eigenvector=np.zeros((92,19))
    i=0
    for f in range(76,92):
        final_eigenvalndex=np.where(eigen_val==sort_eigenval[f])[0]
        final_eigenvector[:,i]=np.asmatrix(eigen_vector[:,final_eigenvalndex]).T  
        i+=1  
    pca_image_matrix_for_training = np.matmul(image_matrix_for_traning,final_eigenvector)    
    return pca_image_matrix_for_training,final_eigenvector,mean_gray,x     

In [7]:
 ###### finding eiven value  and dimension reduce traing dta and mean
pca_image_matrix_for_training,eigenvector,mean_training_data,x=eigenvector_2DPCA(image_name_for_training)

In [8]:
 ###### findind pca based test data ###########
image_matrix_for_testing=np.zeros((len(image_name_for_testing),112,92))
i=0
for name_img in image_name_for_testing :
    img_matrix=cv2.imread(name_img,0)
    img_matrix=img_matrix-mean_training_data
    image_matrix_for_testing[i,:,:]=img_matrix
    i+=1
pca_image_matrix_for_testing=np.matmul(image_matrix_for_testing,eigenvector)   

In [6]:
#### fce recognition #### input is single image it will give recognige fce and corresponding labels
def facerecognigation(img):
    import matplotlib.pyplot as plt
    img_matrix=cv2.imread(img,0)
    img_matrix=img_matrix-mean_training_data
    pca_image_matrix_for_testing=np.matmul(img_matrix,eigenvector)
    matching_matrix=np.square(pca_image_matrix_for_training-pca_image_matrix_for_testing)
    sum_matrix=np.sum(matching_matrix,axis=1)
    abs_matrix=np.abs(np.sum(sum_matrix,axis=1))
    outmax_index=np.array(np.where(abs_matrix==np.min(abs_matrix)))
    img_matrix=cv2.imread(image_name_for_training[outmax_index[0,0]])
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(img_matrix, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray)
    t=labels_name_for_training[outmax_index[0,0]]
    for (x, y, w, h) in faces:
        cv2.rectangle(img_matrix, (x, y), (x+w, y+h), (255, 0, 0), 2)
    cv2.imshow('label = '+str(t),img_matrix)
    cv2.waitKey()
    cv2.destroyAllWindows()
    return labels_name_for_training[outmax_index[0,0]]
    


In [15]:
## for test perpose ###
a=facerecognigation('3.pgm')
a

's11'

In [11]:
#### testing the data for accuracy #############
a=0
for i in range(0,80):
    matching_matrix=np.square(pca_image_matrix_for_training-pca_image_matrix_for_testing[i,:,:])
    sum_matrix=np.sum(matching_matrix,axis=1)
    abs_matrix=np.abs(np.sum(sum_matrix,axis=1))
    outmax_index=np.array(np.where(abs_matrix==np.min(abs_matrix)))
    if (labels_name_for_testing[i]==labels_name_for_training[outmax_index[0,0]]):
        a+=1
    else:
        b=i    
b


8

In [17]:
 ###### haar cascade algorithm for face detection ########
import cv2
# Load the cascade
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
# Read the input image
img = cv2.imread('test.jpg')
# Convert into grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# Detect faces
faces = face_cascade.detectMultiScale(gray)
# Draw rectangle around the faces
for (x, y, w, h) in faces:
    cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
# Display the output
cv2.imshow('img',img)
cv2.waitKey()
cv2.destroyAllWindows()
img.shape

(159, 318, 3)

In [1]:
import cv2

ModuleNotFoundError: No module named 'cv2'